# Imports

In [237]:
import pandas as pd
import numpy as np

# Load Data

In [238]:
data = pd.read_csv('../data/transportation_world.csv', encoding='unicode_escape', sep=';', header=None).drop(0, axis=1)
data.columns = ['Duration', 'Type', 'Year', 'Vacations']
data['Vacations'] = data['Vacations'].apply(lambda x: 0 if x=='..' else int(x))

In [239]:
data

,Duration,Type,Year,Vacations
0,Mindre end fire overnatninger,Bil,2017,1074671
1,Mindre end fire overnatninger,Bil,2018,928717
2,Mindre end fire overnatninger,Bil,2019,779540
3,Mindre end fire overnatninger,Bil,2020,610632
4,Mindre end fire overnatninger,Bil,2021,477843
...,...,...,...,...
107,Mindst fire overnatninger,Andre transportmidler,2019,28908
108,Mindst fire overnatninger,Andre transportmidler,2020,5348
109,Mindst fire overnatninger,Andre transportmidler,2021,3896
110,Mindst fire overnatninger,Andre transportmidler,2022,20914


In [240]:
data_long = data[data['Duration'] == 'Mindst fire overnatninger'].drop('Duration', axis=1)

# Transportation across time and type

In [241]:
data_long = data_long.pivot(index='Year', columns='Type', values='Vacations')
data_long = data_long.reset_index().drop(['Cykel', 'Sejlbåd'], axis=1)
data_long = data_long.rename_axis(None, axis=1)
data_long['Year'] = data_long['Year'].astype(str)
data_long = data_long[['Year','Fly', 'Bil', 'Tog', 'Skib', 'Bus', 'Andre transportmidler']]

In [242]:
data_long = data_long.rename(columns={'Bil': 'Car', 'Tog': 'Train', 'Skib': 'Ship', 'Fly': 'Airplane', 'Andre transportmidler': 'Other forms of transportation'})

In [243]:
transportation_types = list(data_long.columns[1:])
transportation_types

['Airplane', 'Car', 'Train', 'Ship', 'Bus', 'Other forms of transportation']

In [244]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category20


# Ensure numeric types
#for col in data_long.columns[1:]:  # skip 'Year' if it's a string category
#    data_long[col] = pd.to_numeric(data_long[col], errors='coerce')
#print(data_long['Year'].values)
# Categories to stack

# Create the source
source = ColumnDataSource(data_long)

# Set up the figure
p = figure(x_range=data_long["Year"], height=600, width=900, title="Forms of transport for vacation",
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

# Assign colors (Bokeh palettes are limited, so repeat if needed)
colors = Category20[len(transportation_types)]

# Add stacked bars
p.vbar_stack(transportation_types, x='Year', width=0.9, color=colors, source=source, legend_label=transportation_types)

# Style tweaks
p.y_range.start = 0
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None


p.legend.click_policy="mute" ### assigns the click policy (you can try to use ''hide'
p.legend.location = "center"

########################################################
# This line puts the legend outside of the plot area

p.add_layout(p.legend[0], 'left')


# Show the plot
show(p)


In [ ]:
## Can compare to DK where we primarily drive.